<a href="https://colab.research.google.com/github/gayun99/dacon_competition/blob/main/%EC%9E%A1%EC%BC%80%EC%96%B4_lgb_%2B_xgb_%2B_cat_stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from datetime import datetime

In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/jobcare/data/train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/jobcare/data/test.csv")

d_code = pd.read_csv('/content/drive/MyDrive/jobcare/data/속성_D_코드.csv', index_col=0).T.to_dict()
h_code = pd.read_csv('/content/drive/MyDrive/jobcare/data/속성_H_코드.csv', index_col=0).T.to_dict()
l_code = pd.read_csv('/content/drive/MyDrive/jobcare/data/속성_L_코드.csv', index_col=0).T.to_dict()

print("train_data.shape: ", train_data.shape)
print("test_data.shape: ", test_data.shape)

train_data.shape:  (501951, 35)
test_data.shape:  (46404, 34)


In [ ]:
#train_data.head()

In [ ]:
from typing import Dict
def add_code(
    df: pd.DataFrame,
    d_code: Dict[int, Dict[str, int]], 
    h_code: Dict[int, Dict[str, int]], 
    l_code: Dict[int, Dict[str, int]],
) -> pd.DataFrame:
    
    # Copy input data
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    
    return df

train_data = add_code(train_data, d_code, h_code, l_code)
test_data = add_code(test_data, d_code, h_code, l_code)
print("train_data.shape: ", train_data.shape)
print("test_data.shape: ", test_data.shape)

train_data.shape:  (501951, 63)
test_data.shape:  (46404, 62)


In [ ]:
train_data['dt']= pd.to_datetime(train_data['contents_open_dt'], format = "%Y-%m-%d %H:%M:%S")
test_data['dt']= pd.to_datetime(test_data['contents_open_dt'], format = "%Y-%m-%d %H:%M:%S")

In [ ]:
# def preprocessing_contents_open_dt(data):
#     data['contents_open_dt'] = data['contents_open_dt'].astype('str')
#     DATE = data['contents_open_dt'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    
#     DATE = pd.DataFrame(DATE)
#     DATE = DATE.rename(columns = {'contents_open_dt': 'date'})
    
#     DATE['Y'] = DATE['date'].apply(lambda x: x.timetuple()[0])
#     DATE['M'] = DATE['date'].apply(lambda x: x.timetuple()[1])
#     DATE['D'] = DATE['date'].apply(lambda x: x.timetuple()[2])
#     DATE['id'] = data['id']
    
#     data = data.merge(DATE, on = 'id', how = 'left')
#     data = data.drop(columns = ['date', 'contents_open_dt'])
#     return data

# train_data = preprocessing_contents_open_dt(train_data)
# test_data = preprocessing_contents_open_dt(test_data)

# # 안전하게 확인하고 넘어 갑시다. 
# train_data_labels = train_data['target']
# train_data, test_data = train_data.align(test_data, join = 'inner', axis = 1)
# train_data['target'] = train_data_labels
# print("train_data.shape: ", train_data.shape)
# print("test_data.shape: ", test_data.shape)

In [ ]:
#train_data['Y'].value_counts()
#train_data['M'].value_counts()
#train_data['D'].value_counts()

In [ ]:
job_data = train_data.copy()
job_data_test = test_data.copy()

In [ ]:
job_data.dtypes.value_counts()

int64             56
bool               6
object             1
datetime64[ns]     1
dtype: int64

In [ ]:
#job_data.info()

In [ ]:
job_data.select_dtypes('int64').apply(pd.Series.nunique, axis=0)

id                        501951
person_attribute_a             2
person_attribute_a_1           8
person_attribute_b             6
person_prefer_c                5
person_prefer_d_1           1093
person_prefer_d_2           1081
person_prefer_d_3           1043
person_prefer_e               12
person_prefer_f                1
person_prefer_g                1
person_prefer_h_1            279
person_prefer_h_2            279
person_prefer_h_3            279
contents_attribute_i           3
contents_attribute_a           3
contents_attribute_j_1         9
contents_attribute_j           2
contents_attribute_c           4
contents_attribute_k           2
contents_attribute_l        1752
contents_attribute_d        1065
contents_attribute_m           5
contents_attribute_e          12
contents_attribute_h         250
person_rn                 300177
contents_rn               283359
target                         2
person_prefer_d_1_n          443
person_prefer_d_1_s          137
person_pre

In [ ]:
# contents rn과 선지(속성)간의 상관관계?
# 날짜 추가할지
# person_rn 추가할지

In [ ]:
# ! pip install pycaret[full]

In [ ]:
drop = ['id','person_prefer_f', 'person_prefer_g', 'contents_open_dt', 'dt', 'person_rn', 'contents_rn' ]
# contents_rn 때문에 메모리에러 나는거같아서 제외시켜봄

In [ ]:
train = job_data.drop(drop,axis=1)
test = job_data_test.drop(drop,axis=1)

In [ ]:
traina = train.select_dtypes('bool')
trainb = train.select_dtypes('int64')
# trainb= trainb.astype(dtype='int32') #데이터 줄일라고 int64에서 int32으로 변경
#train = pd.concat([traina,trainb],axis=1) 

In [ ]:
cat_features = trainb.columns[ (trainb.nunique() >= 2) & (trainb.nunique() < 50) ].tolist() #bool 제외한 trainb에서 categorical feature추출
target = ['target']
cat_features = list(set(cat_features) - set(target)) #target제외
print(cat_features, len(cat_features))

['person_prefer_d_1_m', 'person_prefer_e', 'contents_attribute_j_1', 'person_attribute_b', 'person_attribute_a', 'person_prefer_h_2_l', 'contents_attribute_c', 'contents_attribute_a', 'contents_attribute_d_l', 'person_prefer_h_3_l', 'contents_attribute_e', 'contents_attribute_m', 'person_prefer_h_1_l', 'contents_attribute_d_m', 'person_prefer_d_3_l', 'contents_attribute_i', 'person_prefer_d_2_l', 'person_prefer_d_2_m', 'contents_attribute_l_l', 'contents_attribute_k', 'contents_attribute_j', 'person_attribute_a_1', 'person_prefer_d_1_l', 'person_prefer_c', 'contents_attribute_h_l', 'person_prefer_d_3_m'] 26


In [ ]:
high_cardinality_features = trainb.columns[ trainb.nunique() >= 50 ].tolist() # 50개 이상인건 high cardinality feature로 변경
print(high_cardinality_features , len(high_cardinality_features))

['person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3', 'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_h', 'person_prefer_d_1_n', 'person_prefer_d_1_s', 'person_prefer_d_2_n', 'person_prefer_d_2_s', 'person_prefer_d_3_n', 'person_prefer_d_3_s', 'contents_attribute_d_n', 'contents_attribute_d_s', 'person_prefer_h_1_m', 'person_prefer_h_2_m', 'person_prefer_h_3_m', 'contents_attribute_h_m', 'contents_attribute_l_n', 'contents_attribute_l_s', 'contents_attribute_l_m'] 24


In [ ]:
cate = list(set().union(cat_features, high_cardinality_features))

In [ ]:
# 순서형
ordinal = ['person_attribute_a_1','person_attribute_b','person_prefer_e','contents_attribute_e']
# 명목형
nominal = list(set(cate) - set(ordinal))

In [ ]:
# # {'salary' : ['low', 'medium', 'high']})
# ordinal_features = {'person_attribute_a_1':['0','1','2','3','4','5','6','7'], 
#                     'person_attribute_b':['0','1','2','3','4','5'], 
#                     'person_prefer_e':['0','1','2','3','4','5','6','7','8','9','10','11'], 
#                     'contents_attribute_e':['0','1','2','3','4','5','6','7','8','9','10','11'] }

In [ ]:
# from pycaret.classification import *
# clf = setup(data = train, target = 'target', categorical_features=cat_features,
#             ordinal_features = ordinal_features,
#             session_id = 42)

In [ ]:
#catboost
! pip install catboost

     |████████████████████████████████| 76.1 MB 1.5 MB/s 


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc

NFOLDS = 5
SEED = 0
NROWS = None

# print('%-55s | %7s | %10s | %10s | %10s' 
#       % ('FEATURES', 'TYPE', 'NB VALUES', 'NB NaNS', 'NaNs (%)'))
# for f_ in data: # .dtypes
#     print("%-55s | %7s | %10s | %10s |    %5.2f"
#           % (f_, str(data[f_].dtype), 
#              str(len(data[f_].value_counts(dropna=False))), 
#              str(data[f_].isnull().sum()),
#              100 * data[f_].isnull().sum() / data.shape[0]
#             )
#          )

# categorical_feats = [
#     f for f in data.columns if data[f].dtype == 'object'
# ]

# for f_ in cate:
#     train[f_], indexer = pd.factorize(train[f_])
#     test[f_] = indexer.get_indexer(test[f_])
    
gc.enable()

train = job_data.drop(drop,axis=1)
test = job_data_test.drop(drop,axis=1)
y_train = train['target']
del train['target']

###################################
# PLEASE DON'T DO THIS AT HOME LOL
# Averaging factorized categorical features defeats my own reasoning
################################### 
# prev_cat_features = [
#     f_ for f_ in prev.columns if prev[f_].dtype == 'object'
# ]
# for f_ in cate:
#     train[f_], _ = pd.factorize(train[f_])
    
# avg_prev = prev.groupby('SK_ID_CURR').mean()
# cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
# avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
# del avg_prev['SK_ID_PREV']

# x_train = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
# x_test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

# x_train = x_train.fillna(0)
# x_test= x_test.fillna(0)

# ntrain = x_train.shape[0]
# ntest = x_test.shape[0]

# excluded_feats = ['SK_ID_CURR']
# features = [f_ for f_ in x_train.columns if f_ not in excluded_feats]

# x_train = x_train[features]
# x_test = x_test[features]

x_train = train.copy()
x_test = test.copy()

#int type category로 변경
# for i in enumerate(cate):
#   ca = i[1]
#   x_train[ca] = x_train[ca].astype('category')
#   x_test[ca] = x_test[ca].astype('category')

ntrain = x_train.shape[0]
ntest = x_test.shape[0]

kf = KFold(n_splits = NFOLDS, shuffle=True, random_state=SEED)

class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]

class CatboostWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]
        
class LightGBMWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['feature_fraction_seed'] = seed
        params['bagging_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]


class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))


def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)


et_params = {
    'n_jobs': -1,
    'n_estimators': 200,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

rf_params = {
    'n_jobs': -1,
    'n_estimators': 200,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.9689,
    'silent': 1,
    'subsample': 0.7281,
    'learning_rate': 0.01936,
    'objective': 'binary:logistic',
    'max_depth': 3,
    'num_parallel_tree': 1,
    'min_child_weight': 618.567,
    'nrounds': 500,
    'num_leaves':14,
    'reg_alpha':58.55,
    'reg_lambda':0.29
}

catboost_params = {
    'iterations': 30,
    'learning_rate': 0.05,
    'max_depth': 10,
#    'l2_leaf_reg': 40,
#    'bootstrap_type': 'Bernoulli',
#    'subsample': 0.8,
#    'scale_pos_weight': 5,
    'eval_metric': 'F1',
#    'od_type': 'Iter',
    'allow_writing_files': False,
    'cat_features' : cate #######
}

lightgbm_params = {
#    'n_estimators':200,
    'learning_rate':0.1,
    'num_leaves':42,
    'colsample_bytree':0.422,
    'subsample':0.925,
    'max_depth':7,
    'reg_alpha':11.7,
    'reg_lambda':75.1,
#    'min_split_gain':0.01,
    'min_child_weight':179.736    
}

xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatboostWrapper(clf= CatBoostClassifier, seed = SEED, params=catboost_params)
lg = LightGBMWrapper(clf = LGBMClassifier, seed = SEED, params = lightgbm_params)

xg_oof_train, xg_oof_test = get_oof(xg)
et_oof_train, et_oof_test = get_oof(et)
rf_oof_train, rf_oof_test = get_oof(rf)
cb_oof_train, cb_oof_test = get_oof(cb)
lg_oof_train, lg_oof_test = get_oof(lg)

print("XG-CV: {}".format(f1_score(y_train, np.round(xg_oof_train).astype(int))))
print("ET-CV: {}".format(f1_score(y_train, np.round(et_oof_train).astype(int))))
print("RF-CV: {}".format(f1_score(y_train, np.round(rf_oof_train).astype(int))))
print("CB-CV: {}".format(f1_score(y_train, np.round(cb_oof_train).astype(int))))
print("LG-CV: {}".format(f1_score(y_train, np.round(lg_oof_train).astype(int))))

x_train = np.concatenate((xg_oof_train,
                          et_oof_train, 
                          rf_oof_train, 
                          cb_oof_train, 
                          lg_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, 
                         et_oof_test, 
                         rf_oof_test, 
                         cb_oof_test,
                         lg_oof_test), axis=1)

print("{},{}".format(x_train.shape, x_test.shape))

logistic_regression = LogisticRegression()
logistic_regression.fit(x_train,y_train)

test['target'] = logistic_regression.predict_proba(x_test)[:,1]

#test[['SK_ID_CURR', 'TARGET']].to_csv('first_submission.csv', index=False, float_format='%.8f')

0:	learn: 0.6049409	total: 2.15s	remaining: 1m 2s
1:	learn: 0.6021437	total: 3.88s	remaining: 54.4s
2:	learn: 0.6084800	total: 5.44s	remaining: 49s
3:	learn: 0.6060773	total: 6.96s	remaining: 45.3s
4:	learn: 0.6130648	total: 8.73s	remaining: 43.7s
5:	learn: 0.6126394	total: 10.2s	remaining: 40.9s
6:	learn: 0.6133983	total: 11.7s	remaining: 38.5s
7:	learn: 0.6164065	total: 13.2s	remaining: 36.4s
8:	learn: 0.6158633	total: 14.8s	remaining: 34.4s
9:	learn: 0.6173562	total: 16.3s	remaining: 32.5s
10:	learn: 0.6184459	total: 17.8s	remaining: 30.8s
11:	learn: 0.6192983	total: 19.3s	remaining: 29s
12:	learn: 0.6192237	total: 20.8s	remaining: 27.2s
13:	learn: 0.6192554	total: 22.3s	remaining: 25.5s
14:	learn: 0.6189455	total: 23.8s	remaining: 23.8s
15:	learn: 0.6191004	total: 25.3s	remaining: 22.2s
16:	learn: 0.6193224	total: 26.8s	remaining: 20.5s
17:	learn: 0.6191936	total: 28.4s	remaining: 18.9s
18:	learn: 0.6198486	total: 29.9s	remaining: 17.3s
19:	learn: 0.6203302	total: 31.4s	remaining: 

In [ ]:
pred = logistic_regression.predict(x_test)
pred

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
submission['target'] = pred
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
submission.to_csv("lgb+xgb+cat+rf+et_stacking.csv",index=False)

In [ ]:
test

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_l,person_prefer_h_1_m,person_prefer_h_2_l,person_prefer_h_2_m,person_prefer_h_3_l,person_prefer_h_3_m,contents_attribute_h_l,contents_attribute_h_m,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l,target
0,True,False,False,True,True,True,1,1,2,1,857,851,1227,4,263,56,49,1,3,10,2,1,2,1147,839,1,5,263,854,853,824,744,847,847,824,744,1227,1227,1227,926,836,831,824,744,250,528,48,366,48,359,250,528,1146,1128,1021,2010,0.489707
1,False,False,False,True,False,False,2,0,2,2,683,1086,662,2,258,263,49,1,3,5,1,1,2,1611,278,1,4,263,683,677,659,618,1086,1086,1053,926,662,660,659,618,276,274,274,216,250,523,250,528,48,359,250,528,1610,1606,1605,2016,0.426063
2,True,False,False,True,True,True,2,3,2,1,514,790,1233,0,177,170,171,3,3,3,1,1,2,1817,490,3,4,177,514,509,482,482,784,783,745,744,1227,1227,1227,926,490,490,482,482,169,453,169,451,169,452,169,453,1812,1811,1810,2020,0.451051
3,True,False,False,True,True,True,1,2,2,5,114,181,175,4,177,170,171,3,3,5,1,1,2,101,150,5,3,177,114,109,56,1,175,152,56,1,175,152,56,1,145,123,56,1,169,453,169,451,169,452,169,453,101,100,99,2006,0.486292
4,True,False,False,True,False,False,1,6,4,5,1082,1078,1056,5,178,177,4,3,1,5,1,1,1,985,1097,1,4,177,1078,1078,1053,926,1078,1078,1053,926,1056,1054,1053,926,1097,1094,1093,926,169,454,169,453,3,316,169,453,984,980,954,2009,0.477890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46399,True,True,True,False,False,False,2,0,4,1,147,46,145,4,2,4,95,3,3,5,1,1,2,759,147,1,5,91,145,123,56,1,46,43,2,1,145,123,56,1,145,123,56,1,1,315,3,316,94,398,85,396,759,759,759,2006,0.681178
46400,True,False,False,True,False,False,2,0,4,1,176,120,159,4,86,31,278,3,3,10,2,1,2,759,147,1,4,91,175,152,56,1,114,109,56,1,159,152,56,1,145,123,56,1,85,391,30,342,277,542,85,396,759,759,759,2006,0.502354
46401,True,True,False,True,True,True,2,0,3,1,145,46,147,5,288,279,278,3,3,10,2,1,2,759,147,1,4,288,145,123,56,1,46,43,2,1,145,123,56,1,145,123,56,1,277,547,277,543,277,542,277,547,759,759,759,2006,0.640289
46402,True,True,False,True,True,True,1,3,3,2,145,46,147,6,288,279,278,3,1,5,1,3,2,759,147,1,5,288,145,123,56,1,46,43,2,1,145,123,56,1,145,123,56,1,277,547,277,543,277,542,277,547,759,759,759,2006,0.667035


In [ ]:
pred = np.round(test['target']).astype(int)
pred

0        0
1        0
2        0
3        0
4        0
        ..
46399    1
46400    1
46401    1
46402    1
46403    1
Name: target, Length: 46404, dtype: int64

In [ ]:
ntrain = x_train.shape[0]
ntrain

501951

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split


In [ ]:
X = train.copy()
X = X.drop(['target'],axis=1)
target = train['target']

#int type category로 변경
# for i in enumerate(cate):
#     ca = i[1]
#     X[ca] = X[ca].astype('category')

X_train,X_test,y_train,y_test = train_test_split(X,target,
                                                 test_size=0.2,random_state=42, stratify = target)

In [ ]:
#X_train.info()

In [ ]:
def learning_rate_010_decay_power_099(current_iter):
  base_learning_rate = 0.1
  lr = base_learning_rate  * np.power(.99, current_iter)
  return lr if lr > 1e-3 else 1e-3

import lightgbm as lgb

# fit_params={'early_stopping_rounds':30, 
#             'metric' : 'custom', 
#             "eval_set" : [(X_test,y_test)],
#             'eval_names': ['valid'],
#             'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
#             'verbose': 100,
#             'categorical_feature': 'auto'}

#set up hyperparameter search
# from scipy.stats import randint as sp_randint
# from scipy.stats import uniform as sp_uniform
# param_test ={'num_leaves': list(range(6,50,3)),
#              'min_child_samples': list(range(100,500,15)), 
#              'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
#              'subsample': [0.2,0.4,0.6,0.8,0.9], 
#              'colsample_bytree': [0.6,0.7,0.8,0.9], 
#              'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
#              'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}



In [ ]:
pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=5f8ec5f77f5c241484b7c7e8369ffd4cc55e69ac0000c7f2739792df55e50537
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [ ]:
# lgbm / metric = f1-score 

import lightgbm as lgb
from sklearn.metrics import f1_score
from bayes_opt import BayesianOptimization
n_folds=5
d_train = lgb.Dataset(X_train, label=y_train, free_raw_data=False) #categorical feature 수정해볼것 - contents_rn 포함, categorical data 제외

# def lgb_accuracy(y_hat, data):
#     y_true = data.get_label().astype(int)
#     y_hat = np.around(y_hat).astype(int) # pred반올림
#     return 'accuracy', accuracy_score(y_true, y_hat), True
param_test ={'num_leaves': (6,45),
             'min_child_weight': (1e-3, 1e3), #0.001 ~ 1000
             'subsample': (0.5,1), 
             'colsample_bytree': (0.2,1), 
             'reg_alpha': (0, 100),
             'reg_lambda': (0, 100),
             'learning_rate': (0.01,0.2),
             'max_depth': (3,8)}

             
# fit_params={'early_stopping_rounds':30, 
#             'metric' : 'custom', 
#             "eval_set" : [(X_test,y_test)],
#             'eval_names': ['valid'],
#             'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
#             'verbose': 100,
#             'categorical_feature': 'auto'
#             'max_depth :(3,7)'}

# def accuracy(y_pred, data):
#     y_true = np.array(data.get_label().astype(int))
#     y_pred = (y_pred >= 0.5)
#     score = accuracy_score(y_true, y_pred)

#     return 'accuracy', score, True

def lgb_f1_score(y_hat, data):
    y_true = data.get_label().astype(int)
#    y_hat = (y_hat >= 0.5)
    y_hat = np.round(y_hat).astype(int) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True


# def learning_rate_010_decay_power_099(current_iter):
#   base_learning_rate = 0.1
#   lr = base_learning_rate  * np.power(.99, current_iter)
#   return lr if lr > 1e-3 else 1e-3    

def lgb_cv(num_leaves,min_child_weight,subsample,colsample_bytree,reg_alpha,reg_lambda,max_depth,learning_rate):
    params = {'num_iterations' : 5000, 
             'early_stopping_rounds':30, #30에서 변경
             'n_jobs':-1,
              'metric' : 'custom', #metric 수정
             }
    params['num_leaves'] = int(round(num_leaves))
    params['min_child_weight'] = min_child_weight
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample_bytree
    params['reg_alpha'] = reg_alpha
    params['reg_lambda'] = reg_lambda
    params['learning_rate'] = learning_rate
    params['max_depth']= int(round(max_depth))
    lgbcv = lgb.cv(params, d_train, nfold=n_folds, seed=42, stratified=True, verbose_eval = 200,
                   metrics=['None'], feval=lgb_f1_score)
    
    return max(lgbcv['f1-mean'])

    
lgbBO = BayesianOptimization(lgb_cv,param_test,random_state=42,verbose=2)
init_round=5
opt_round = 10
lgbBO.maximize(init_points=init_round, n_iter=opt_round) # n_iter 찾아보기?
print(lgbBO.max)
# params = lgbBO.res['max']['max_params']
# lgb2 = lgb.train(params, X_train , 100)
# lgb2 = lgb.train(params, d_train, 100)
# lgb_prob = lgb2.predict( y_test.values )

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:430: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:435: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  1        |  0.6234   |  0.4996   |  0.1906   |  6.66     |  598.7    |  12.08    |  15.6     |  5.808    |  0.9331   |


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:430: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:435: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


KeyboardInterrupt: ignored

In [ ]:
#lgbBO.res['max']['max_params']
#lgbBO
#params = {'colsample_bytree': 0.71, 'learning_rate': 0.095, 'max_depth': 6, 'min_child_weight': 703.52, 'num_leaves': 39, 'reg_alpha': 64.05, 'reg_lambda': 73.95, 'subsample': 0.74}
#params = {'colsample_bytree': 0.24, 'min_child_weight': 866, 'num_leaves': 29, 'reg_alpha': 35.40, 'reg_lambda': 1.0, 'subsample': 0.94}
lgb_params = {'colsample_bytree': 0.422, 'learning_rate': 0.1048, 'max_depth': 7, 'min_child_weight': 179.736, 'num_leaves': 42, 'reg_alpha': 11.704, 'reg_lambda': 75.106, 'subsample': 0.925}

In [ ]:
a = {'num_iterations' : 5000, 'early_stopping_rounds':100, 'n_jobs':-1, 'metric' : 'custom', 'verbose': 100, 'seed' : 42} #원래 있던 파라미터들 추가, early_stopping만 100으로 변경
lgb_params.update(a)
lgb2 = lgb.train(lgb_params, d_train, valid_sets= lgb.Dataset(X_test,y_test), feval=lgb_f1_score)

#lgb_prob = lgb2.predict( X_test )

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's f1: 0.571561
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's f1: 0.616071
[3]	valid_0's f1: 0.615473
[4]	valid_0's f1: 0.610438
[5]	valid_0's f1: 0.603898
[6]	valid_0's f1: 0.605951
[7]	valid_0's f1: 0.599532
[8]	valid_0's f1: 0.609857
[9]	valid_0's f1: 0.610256
[10]	valid_0's f1: 0.612791
[11]	valid_0's f1: 0.61609
[12]	valid_0's f1: 0.615517
[13]	valid_0's f1: 0.61666
[14]	valid_0's f1: 0.617322
[15]	valid_0's f1: 0.618314
[16]	valid_0's f1: 0.618931
[17]	valid_0's f1: 0.617654
[18]	valid_0's f1: 0.617614
[19]	valid_0's f1: 0.619387
[20]	valid_0's f1: 0.619114
[21]	valid_0's f1: 0.619787
[22]	valid_0's f1: 0.619757
[23]	valid_0's f1: 0.619947
[24]	valid_0's f1: 0.620223
[25]	valid_0's f1: 0.62063
[26]	valid_0's f1: 0.621051
[27]	valid_0's f1: 0.621965
[28]	valid_0's f1: 0.621336
[29]	valid_0's f1: 0.621678
[30]	valid_0's f1: 0.621763
[31]	valid_0's f1: 0.622342
[32]	valid_0's f1: 0.622611
[33]	valid_0's f1: 0.623039
[34]	valid_0's f1: 0.6234

In [ ]:
lgb_prob = lgb2.predict( test )
ypred = np.around(lgb_prob).astype(int)
#print(ypred)

In [ ]:
# xgboost / metric = f1-score 

import xgboost as xgb
from sklearn.metrics import f1_score
from bayes_opt import BayesianOptimization
n_folds=5
d_train = xgb.DMatrix(X_train, y_train)
d_val = xgb.DMatrix(X_test,y_test)

param_test ={'num_leaves': (6,45),
             'min_child_weight': (1e-3, 1e3), #0.001 ~ 1000
             'subsample': (0.5,1), 
             'colsample_bytree': (0.2,1), 
             'reg_alpha': (0, 100),
             'reg_lambda': (0, 100),
             'learning_rate': (0.01,0.2),
             'max_depth': (3,8)}

def xgb_f1_score(y_hat, data):
    y_true = data.get_label().astype(int)
#    y_hat = (y_hat >= 0.5)
    y_hat = np.round(y_hat).astype(int) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat)


def xgb_cv(num_leaves,min_child_weight,subsample,colsample_bytree,reg_alpha,reg_lambda,max_depth,learning_rate):
    params = {'nrounds' : 5000, #num_iteration
             'early_stopping_rounds':30, #30에서 변경
             'n_jobs':-1
#              'metric' : 'custom', #xgb랑 lgb랑 다름
             }
    params['num_leaves'] = int(round(num_leaves))
    params['min_child_weight'] = min_child_weight
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample_bytree
    params['reg_alpha'] = reg_alpha
    params['reg_lambda'] = reg_lambda
    params['learning_rate'] = learning_rate
    params['max_depth']= int(round(max_depth))
    xgbcv = xgb.cv(params, d_train, nfold=n_folds, seed=42, stratified=True, verbose_eval = 200,
                    feval=xgb_f1_score)
    
    mean_f1 = xgbcv["test-f1-mean"].values[-1]
    
    return mean_f1
  
lgbB1 = BayesianOptimization(xgb_cv,param_test,random_state=42,verbose=2)
init_round=5
opt_round = 10
lgbB1.maximize(init_points=init_round, n_iter=opt_round) # n_iter 찾아보기?
print(lgbB1.max)



|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
[0]	train-f1:0.57716+0.0105127	train-rmse:0.496479+3.68054e-05	test-f1:0.573663+0.00963472	test-rmse:0.496566+6.24083e-05


KeyboardInterrupt: ignored

In [ ]:
xgb_params={'colsample_bytree': 0.9689, 'learning_rate': 0.01936, 'max_depth': 3, 'min_child_weight': 618.567, 'num_leaves': 14, 'reg_alpha': 58.5531, 'reg_lambda': 0.2868, 'subsample': 0.7281}

In [ ]:
a = {'num_iterations' : 5000, 'early_stopping_rounds':100, 'n_jobs':-1, 'metric' : 'custom', 'verbose': 100, 'seed' : 42} #원래 있던 파라미터들 추가, early_stopping만 100으로 변경
xgb_params.update(a)
d_train = xgb.DMatrix(X_train, y_train)
d_test = xgb.DMatrix(X_test,y_test)
watch_list = [(d_test, 'eval'), (d_train, 'train')]
xgb2 = xgb.train(xgb_params, d_train, evals = watch_list, feval=xgb_f1_score)

[0]	eval-rmse:0.499661	train-rmse:0.499677	eval-f1:0.642685	train-f1:0.641215
[1]	eval-rmse:0.49934	train-rmse:0.499347	eval-f1:0.642678	train-f1:0.641199
[2]	eval-rmse:0.499057	train-rmse:0.499071	eval-f1:0.642678	train-f1:0.641199
[3]	eval-rmse:0.49874	train-rmse:0.498713	eval-f1:0.642678	train-f1:0.641199
[4]	eval-rmse:0.498468	train-rmse:0.498439	eval-f1:0.642678	train-f1:0.641199
[5]	eval-rmse:0.498152	train-rmse:0.498232	eval-f1:0.642678	train-f1:0.641199
[6]	eval-rmse:0.497941	train-rmse:0.497899	eval-f1:0.642678	train-f1:0.641199
[7]	eval-rmse:0.497655	train-rmse:0.497588	eval-f1:0.642678	train-f1:0.641199
[8]	eval-rmse:0.497402	train-rmse:0.497304	eval-f1:0.642678	train-f1:0.641199
[9]	eval-rmse:0.497192	train-rmse:0.497114	eval-f1:0.641011	train-f1:0.639746


In [ ]:
#test data 변경

#int type category로 변경
# for i in enumerate(cate):
#     ca = i[1]
#     test[ca] = test[ca].astype('category') 

#test데이타 예측
dtest = xgb.DMatrix(test)
xgb_prob = xgb2.predict( dtest )

In [ ]:
xgb_prob # predict 한게 저렇게 나오는게 맞나..? 0.5이상이면 1, 아님 0 해야겟지..?

array([0.50657856, 0.48306903, 0.47263995, ..., 0.5199679 , 0.5199679 ,
       0.5199679 ], dtype=float32)

In [ ]:
y_pred = np.around(xgb_prob).astype(int)

In [ ]:
print(y_pred)

[1 0 0 ... 1 1 1]


In [ ]:
#catboost
! pip install catboost

     |████████████████████████████████| 76.1 MB 1.1 MB/s 


In [ ]:
from catboost import Pool, CatBoostClassifier
import catboost as cb
model_cb = CatBoostClassifier(task_type='CPU', iterations = 150,
                              random_state = 42, 
                              eval_metric="F1")

In [ ]:
model_cb.fit(X_train, y_train, cat_features= cate, plot=True, 
             eval_set=(X_test, y_test))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.318006
0:	learn: 0.6250642	test: 0.6272690	best: 0.6272690 (0)	total: 1.45s	remaining: 3m 36s
1:	learn: 0.6075590	test: 0.6048413	best: 0.6272690 (0)	total: 2.75s	remaining: 3m 23s
2:	learn: 0.6129553	test: 0.6096984	best: 0.6272690 (0)	total: 3.97s	remaining: 3m 14s
3:	learn: 0.6145074	test: 0.6107206	best: 0.6272690 (0)	total: 4.86s	remaining: 2m 57s
4:	learn: 0.6148459	test: 0.6117961	best: 0.6272690 (0)	total: 5.75s	remaining: 2m 46s
5:	learn: 0.6179860	test: 0.6164480	best: 0.6272690 (0)	total: 6.7s	remaining: 2m 40s
6:	learn: 0.6184084	test: 0.6167516	best: 0.6272690 (0)	total: 7.61s	remaining: 2m 35s
7:	learn: 0.6198116	test: 0.6174293	best: 0.6272690 (0)	total: 8.55s	remaining: 2m 31s
8:	learn: 0.6195517	test: 0.6168640	best: 0.6272690 (0)	total: 9.45s	remaining: 2m 28s
9:	learn: 0.6208182	test: 0.6194179	best: 0.6272690 (0)	total: 10.4s	remaining: 2m 24s
10:	learn: 0.6217102	test: 0.6210618	best: 0.6272690 (0)	total: 11.3s	remaining: 2m 22s
11:	learn: 0.

In [ ]:
cat_prob = model_cb.predict_proba( test )
cat_prob

array([[0.59233736, 0.40766264],
       [0.54435859, 0.45564141],
       [0.57676357, 0.42323643],
       ...,
       [0.40866653, 0.59133347],
       [0.32985868, 0.67014132],
       [0.34732056, 0.65267944]])

In [ ]:
pred = model_cb.predict(test)
pred

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
#np.round(0.49931039).astype(int)
prob=pd.DataFrame(ypred)
prob[1] = pd.DataFrame(y_pred)
prob[2] = pd.DataFrame(pred)
prob['avg'] = np.round(prob.mean(axis=1)).astype(int)
prob.head(10)

,0,1,2,avg
0,1,1,0,1
1,0,0,0,0
2,0,0,0,0
3,0,1,0,0
4,1,1,1,1
5,0,0,0,0
6,0,0,1,0
7,0,0,0,0
8,1,1,1,1
9,0,0,0,0


In [ ]:
# from sklearn.ensemble import VotingClassifier
# estimators=[('lgbm', lgb2), ('xgb', xgb2), ('catboost', model_cb)]
# vclf = VotingClassifier(estimators=estimators)
# vclf2 = vclf.fit(X_train, y_train )

# from sklearn.ensemble import StackingClassifier
# estimators=[('lgbm', lgb2), ('xgb', xgb2)]
# sclf = StackingClassifier(estimators=estimators, final_estimator=model_cb)
# sclf2 = sclf.fit(X_train, y_train )

In [ ]:
#np.round(0.49931039).astype(int)

In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/jobcare/data/sample_submission.csv")
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
submission['target'] = prob['avg']
submission.head()

,id,target
0,0,1
1,1,0
2,2,0
3,3,0
4,4,1


In [ ]:
submission.to_csv("lgb+xgb+cat_mean.csv",index=False)

In [ ]:
# from sklearn.metrics import accuracy_score
# score = accuracy_score(y_test, y_pred)
# print(score)

In [ ]:
# gs.fit(X_train, y_train, **fit_params)

In [ ]:
# print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))